In [0]:
%run /Workspace/Users/loganathansundaramsa@gmail.com/dataengg_inc_learning/databricks_learning/Brazilian_Ecommerce/gendral_config/com_utl_fun


In [0]:

import json

gendral_config = dbutils.notebook.run("/Workspace/Users/loganathansundaramsa@gmail.com/dataengg_inc_learning/databricks_learning/Brazilian_Ecommerce/gendral_config/common_notebook", 120)


config_dict = json.loads(gendral_config)

src = config_dict['src']
bronze = config_dict['bronze']
silver = config_dict['silver']
gold = config_dict['gold']
catalog = config_dict['CATALOG']
schema = config_dict['SCHEMA']

In [0]:
df_payments = read_file(spark,"parquet",f'{bronze}/order_payments')

columns = {
        "payment_value": "decimal(10,2)",
        "payment_installments":"integer"}

df_cast = f_cast_col(df_payments, columns)

df_dedup = deduplicate(df_cast,["order_id", "payment_sequential"])

df_dropnull = drop_required_nulls(df_dedup,["order_id","payment_value"]) \
    .filter("payment_value > 0").filter("payment_installments>=0")

colm = {"payment_type":"lower_trim"}
df_standardize = standardize_string_columns(df_dropnull,colm)

columns = {
        "created_at": {"type": "timestamp"},
        "updated_at": {"type": "timestamp"},
        "source": {"type": "string", "default": "payments"}}

df_final_payments = enrich_add_col(df_standardize, columns)

In [0]:
# Write data delta file /delta table in silver location

write_file(df_final_payments,f"{silver}/silver_payments")

write_delta_liquid(df_final_payments, f"{catalog}.{schema}.silver_payments",["order_id"])